## LLM PEFT BERT with Context Relavant Co



In [7]:
# Let's import the public training set and take a look
import pandas as pd

train_df = pd.read_csv('./data/train.csv')
print(train_df.shape)
train_df.head()

(200, 8)


,id,prompt,A,B,C,D,E,answer
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D


In [8]:
train_pages_df = pd.read_csv('./data/train_pages_df.csv')
phy_pages_df = pd.read_csv('./data/physics_pages_df.csv')

In [9]:
train_pages_df=pd.concat([train_pages_df,phy_pages_df],axis=0)

In [10]:
train_pages_df

,page,section_title,text,cleaned_text,chunk_id
0,Supersymmetric quantum mechanics,Introduction,Understanding the consequences of supersymmetr...,understanding consequences supersymmetry susy ...,1
1,Supersymmetric quantum mechanics,Example: the harmonic oscillator,The Schrödinger equation for the harmonic osci...,schr dinger equation harmonic oscillator takes...,1
2,Supersymmetric quantum mechanics,Example: the harmonic oscillator,The Schrödinger equation for the harmonic osci...,"special case shape invariance, discussed below...",2
3,Supersymmetric quantum mechanics,SUSY QM superalgebra,"In fundamental quantum mechanics, we learn tha...","fundamental quantum mechanics, learn algebra o...",1
4,Supersymmetric quantum mechanics,Example,Let's look at the example of a one-dimensional...,let's look example one dimensional nonrelativi...,1
...,...,...,...,...,...
4503,Equipartition theorem,General formulation of the equipartition theorem,The most general form of the equipartition the...,k.,2
4504,Equipartition theorem,Summary,"In classical statistical mechanics, the equipa...","classical statistical mechanics, equipartition...",1
4505,Quartz crystal microbalance with dissipation m...,Method,Ring-down as a method to interrogate acoustic ...,ring method interrogate acoustic resonators es...,1
4506,Quartz crystal microbalance with dissipation m...,Mass sensor,"When operated as a mass sensor, the QCM-D is o...","operated mass sensor, qcm often used study mol...",1


In [11]:
# For convenience we'll turn our pandas Dataframe into a Dataset
from datasets import Dataset
train_ds = Dataset.from_pandas(train_df)

In [12]:
train_ds

Dataset({
    features: ['id', 'prompt', 'A', 'B', 'C', 'D', 'E', 'answer'],
    num_rows: 200
})

In [13]:
train_ds[0]

{'id': 0,
 'prompt': 'Which of the following statements accurately describes the impact of Modified Newtonian Dynamics (MOND) on the observed "missing baryonic mass" discrepancy in galaxy clusters?',
 'A': 'MOND is a theory that reduces the observed missing baryonic mass in galaxy clusters by postulating the existence of a new form of matter called "fuzzy dark matter."',
 'B': 'MOND is a theory that increases the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 20.',
 'C': 'MOND is a theory that explains the missing baryonic mass in galaxy clusters that was previously considered dark matter by demonstrating that the mass is in the form of neutrinos and axions.',
 'D': 'MOND is a theory that reduces the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 2.',
 'E': 'MOND 

In [14]:
from transformers import AutoTokenizer
import pprint
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [15]:
# Now we'll instatiate the model that we'll finetune on our public dataset, then use to
# make prediction on the private dataset.
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
import torch
model = AutoModelForMultipleChoice.from_pretrained('bert-base-uncased')

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
import random
import pandas as pd
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt_tab')
                    
def prepare_nsp_data(corpus, max_samples=None):
    # Step 1: Split the corpus into sentences
    sentences = sent_tokenize(corpus)
    
    # Step 2: Create positive pairs (consecutive sentences)
    positive_pairs = [(sentences[i], sentences[i+1], 1) for i in range(len(sentences) - 1)]
    
    # Step 3: Create negative pairs (randomly picked second sentence)
    negative_pairs = []
    for i in range(len(sentences) - 1):
        sentence_A = sentences[i]
        random_sentence_B = random.choice(sentences)
        
        # Ensure sentence_B is not the actual next sentence
        while random_sentence_B == sentences[i + 1]:
            random_sentence_B = random.choice(sentences)
            
        negative_pairs.append((sentence_A, random_sentence_B, 0))
    
    # Combine positive and negative pairs
    dataset = positive_pairs + negative_pairs
    
    # Step 4: Optionally, limit the number of samples
    if max_samples:
        dataset = random.sample(dataset, max_samples)
    
    # Convert to a DataFrame for easy saving or manipulation
    df = pd.DataFrame(dataset, columns=["sentence_A", "sentence_B", "label"])
    
    return df

dfs=[]
for i in range(train_pages_df.shape[0]):
    df=prepare_nsp_data(train_pages_df['cleaned_text'].iloc[i])
    dfs.append(df)
combined_df = pd.concat(dfs, ignore_index=True)    
# Saving the dataset to a .csv file for BERT NSP training
#df.to_csv("nsp_training_data.csv", sep="\t", index=False)
print(combined_df.shape)
combined_df.head()

[nltk_data] Downloading package punkt_tab to /home/vino/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


(89074, 3)


,sentence_A,sentence_B,label
0,understanding consequences supersymmetry susy ...,"make progress problems, physicists developed s...",1
1,"make progress problems, physicists developed s...",hoped studying susy's consequences simpler set...,1
2,hoped studying susy's consequences simpler set...,"example, students typically taught ""solve"" hyd...",1
3,"example, students typically taught ""solve"" hyd...",considerable amount work using many differenti...,1
4,considerable amount work using many differenti...,outcome spectrum hydrogen atom energy states l...,1


In [18]:
import torch
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Custom Dataset class for BERT NSP task
class BertNSPDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # Get sentence A, sentence B, and label
        sentence_A = self.data.iloc[index]['sentence_A']
        sentence_B = self.data.iloc[index]['sentence_B']
        label = self.data.iloc[index]['label']
        
        # Encode sentence pair using BERT tokenizer
        encoding = self.tokenizer.encode_plus(
            sentence_A,
            sentence_B,
            add_special_tokens=True,       # Add [CLS], [SEP]
            max_length=self.max_len,       # Maximum length of the input sequence
            return_token_type_ids=True,    # Necessary for sentence pairs
            padding='max_length',          # Pad up to the max length
            truncation=True,               # Truncate sentences to the max length
            return_attention_mask=True,    # Create attention mask
            return_tensors='pt'            # Return PyTorch tensors
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)  # Convert label to tensor
        }

# Function to prepare the dataset
def create_nsp_dataset(df, tokenizer, max_len=128):
    return BertNSPDataset(df, tokenizer, max_len)

# Example of using the dataset
max_len = 128  # Max length of the input sequence

# Create the dataset
nsp_dataset = create_nsp_dataset(combined_df, tokenizer, max_len)

nsp_dataset_mini = create_nsp_dataset(combined_df.iloc[:128], tokenizer, max_len)

print('len of dataset:',len(nsp_dataset))
# Optionally, use DataLoader for batching
dataloader = DataLoader(nsp_dataset, batch_size=32, shuffle=True)


len of dataset: 89074


In [24]:
from transformers import BertForNextSentencePrediction, BertTokenizer, Trainer, TrainingArguments
from peft import PeftConfig, PeftModel
from peft import get_peft_model, LoraConfig, TaskType
import torch

# Initialize BERT with Next Sentence Prediction head
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Prepare PEFT configuration (for LoRA in this case)
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # NSP task
    r=8,                                         # Low-rank factor
    lora_alpha=32,                               # Scaling factor
    lora_dropout=0.1,                            # Dropout for LoRA
    target_modules=["query", "value"],           # Target attention layers
)

# Apply LoRA to the model
peft_model = get_peft_model(model, peft_config)

# Verify the trainable parameters
peft_model.print_trainable_parameters()

# Training arguments
training_args = TrainingArguments(
    output_dir="./data/peft/",       # Output directory
    evaluation_strategy="epoch",  # Evaluate at the end of every epoch
    learning_rate=1e-4,           # Learning rate
    per_device_train_batch_size=32,   # Batch size for training
    per_device_eval_batch_size=32,    # Batch size for evaluation
    num_train_epochs=30,           # Number of epochs
    weight_decay=0.001,            # Weight decay
    logging_dir='./logs',         # Directory for logging
    logging_steps=100,
)

# DataLoader already prepared
# `nsp_dataset` should be prepared as discussed in the previous steps

# Initialize the Trainer
trainer = Trainer(
    model=peft_model,             # PEFT BERT model with LoRA
    args=training_args,           # Training arguments
    train_dataset=nsp_dataset_mini,    # Training dataset
    eval_dataset=nsp_dataset_mini,     # Evaluation dataset (you can split the data if needed)
    tokenizer=tokenizer           # Tokenizer
)




trainable params: 294,912 || all params: 109,778,690 || trainable%: 0.2686


/home/vino/anaconda3/envs/py311/lib/python3.11/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_24645/3059379090.py:44: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [25]:
# Start training
trainer.train()

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Epoch,Training Loss,Validation Loss
1,No log,6.223266
2,No log,6.140319
3,No log,6.021801
4,No log,5.803170
5,No log,5.535934
6,No log,5.132133
7,No log,4.541801
8,No log,3.539258
9,No log,2.212578
10,No log,1.217492


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

TrainOutput(global_step=400, training_loss=0.9986552715301513, metrics={'train_runtime': 70.345, 'train_samples_per_second': 181.96, 'train_steps_per_second': 5.686, 'total_flos': 844854480076800.0, 'train_loss': 0.9986552715301513, 'epoch': 100.0})

In [26]:
model.save_pretrained("./data/peft/model/", safe_serialization=True)
tokenizer.save_pretrained("./data/peft/model/")

('./data/peft/model/tokenizer_config.json',
 './data/peft/model/special_tokens_map.json',
 './data/peft/model/vocab.txt',
 './data/peft/model/added_tokens.json')

In [34]:
### model trained in GPU. Load the trained model
### only 2 training epochs are done 
### to show model improvement over plain-BERT
model = AutoModelForMultipleChoice.from_pretrained('./data/peft/model/')
tokenizer = AutoTokenizer.from_pretrained('./data/peft/model/')

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at ./data/peft/model/ and are newly initialized: ['bert.encoder.layer.0.attention.self.query.bias', 'bert.encoder.layer.0.attention.self.query.weight', 'bert.encoder.layer.0.attention.self.value.bias', 'bert.encoder.layer.0.attention.self.value.weight', 'bert.encoder.layer.1.attention.self.query.bias', 'bert.encoder.layer.1.attention.self.query.weight', 'bert.encoder.layer.1.attention.self.value.bias', 'bert.encoder.layer.1.attention.self.value.weight', 'bert.encoder.layer.10.attention.self.query.bias', 'bert.encoder.layer.10.attention.self.query.weight', 'bert.encoder.layer.10.attention.self.value.bias', 'bert.encoder.layer.10.attention.self.value.weight', 'bert.encoder.layer.11.attention.self.query.bias', 'bert.encoder.layer.11.attention.self.query.weight', 'bert.encoder.layer.11.attention.self.value.bias', 'bert.encoder.layer.11.attention.self.value.weight', 'bert.encoder.layer.2.attention.self.quer

In [35]:
# We'll create a dictionary to convert option names (A, B, C, D, E) into indices and back again
options = 'ABCDE'
indices = list(range(5))

option_to_index = {option: index for option, index in zip(options, indices)}
index_to_option = {index: option for option, index in zip(options, indices)}

def preprocess(example):
    # The AutoModelForMultipleChoice class expects a set of question/answer pairs
    # so we'll copy our question 5 times before tokenizing
    first_sentence = [example['prompt']] * 5
    
    second_sentence = []
    for option in options:
        second_sentence.append(example[option])
    
    # Our tokenizer will turn our text into token IDs BERT can understand
    tokenized_example = tokenizer(first_sentence, second_sentence, padding='max_length', truncation=True, max_length=128)
    tokenized_example['label'] = option_to_index[example['answer']]

    return tokenized_example

tokenized_train_ds = train_ds.map(preprocess, batched=False, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [36]:
# Function to make predictions
def predict_mcq(example, model):
    predictions = []
    
    # Extract necessary inputs from dataset
    input_ids = torch.tensor(example['input_ids']).unsqueeze(0)  # Add batch dimension
    attention_mask = torch.tensor(example['attention_mask']).unsqueeze(0)
    token_type_ids = torch.tensor(example['token_type_ids']).unsqueeze(0)
        
    # Get predictions
    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
    logits = outputs.logits
    predicted_index = torch.argmax(logits, dim=1).item()
    
    return predicted_index

# Run prediction over the dataset
pred_idx=[]
for data in tokenized_train_ds:
     pred_idx.append(predict_mcq(data , model))

In [37]:
# Get predicted answer
option_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E'}
prediction = [option_mapping[i] for i in pred_idx]

In [38]:
train_df['predicted_answer']=prediction

In [39]:
count=0
for i in range(train_df.shape[0]):
    if (train_df['answer'].iloc[i]==train_df['predicted_answer'].iloc[i]):
        count=count+1
print(count/200)

0.245


### BERT with PEFT (context data) improves from 0.18 to 0.28
#### context relavant corpus data converted to NSP daatset. Model only trained for 2 epochs on NSP style daatset. Further training may improve the accuracy.